# An implementation of the K-means - exercise

In this exercise implement the K-means algorithm for unsupervised learning.

## 1. Loading the dataset

In [1]:
# loading the libraries
import numpy as np
import math 
import pylab as pil
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
# loading the dataset
data = pd.read_csv('ecoli.csv', header = None).dropna()
data.tail(10)

,0,1,2,3,4,5,6,7,8
326,SUBI_ECOLI,0.62,0.83,0.48,0.5,0.46,0.36,0.40,pp
327,TBPA_ECOLI,0.56,0.54,0.48,0.5,0.43,0.37,0.30,pp
328,TESA_ECOLI,0.69,0.66,0.48,0.5,0.41,0.50,0.25,pp
329,TOLB_ECOLI,0.69,0.65,0.48,0.5,0.63,0.48,0.41,pp
330,TORA_ECOLI,0.43,0.59,0.48,0.5,0.52,0.49,0.56,pp
331,TREA_ECOLI,0.74,0.56,0.48,0.5,0.47,0.68,0.30,pp
332,UGPB_ECOLI,0.71,0.57,0.48,0.5,0.48,0.35,0.32,pp
333,USHA_ECOLI,0.61,0.60,0.48,0.5,0.44,0.39,0.38,pp
334,XYLF_ECOLI,0.59,0.61,0.48,0.5,0.42,0.42,0.37,pp
335,YTFQ_ECOLI,0.74,0.74,0.48,0.5,0.31,0.53,0.52,pp


In [3]:
from sklearn.preprocessing import LabelEncoder
le =LabelEncoder()
y = data[8]
y = le.fit_transform(y)
np.unique(y)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [4]:
# dropping the first column
data = data.iloc[:,data.columns!=0]
data.head()

,1,2,3,4,5,6,7,8
0,0.49,0.29,0.48,0.5,0.56,0.24,0.35,cp
1,0.07,0.40,0.48,0.5,0.54,0.35,0.44,cp
2,0.56,0.40,0.48,0.5,0.49,0.37,0.46,cp
3,0.59,0.49,0.48,0.5,0.52,0.45,0.36,cp
4,0.23,0.32,0.48,0.5,0.55,0.25,0.35,cp


In [5]:
# selecting only the first 7 columns
X = data.iloc[:,:7].values

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

In [7]:
# Now, proababy I should also define the X_std
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)

## 2. Implementing the Kmeans algorithm.

In [8]:
# Import the necessary libraries
import numpy as np
from numpy import linalg 

# Let us define the L1 distance for later
def L1(v1,v2):
      if(len(v1)!=len(v2)):
        print('error')
        return -1
      return np.sum([abs(v1[i]-v2[i]) for i in range(len(v1))])


class KMeans_custom(object):
     
    # Initialization of the K-means object
    def __init__(self, metric, tol, random_seed=None):
        self.tol = tol
        self.metric = metric
        self.random_seed = random_seed
    
    def fit(self, data, num_clusters):
        # K-means algorithm
        # Initialize the parameters
        self.K = num_clusters
        
        # Initialize random seed 
        if(self.random_seed == None):
            np.random.seed(26)
        else:
            np.random.seed(self.random_seed)
        
        # Initialize mean vectors (one for each cluster) (KxN)
        self.mu = np.random.rand(self.K,data.shape[1])
        #print('mu:',self.mu)
        
        self.errors =[]
        # the iteration loop        
        while True:
            
            # Now initialize the array C. It contains the 
            self.C = [[] for i in range(self.K)]
            
            # Decide which cluster the data belongs to
            for i_data in range(data.shape[0]):
                
                distances_sample = []
                x = data[i_data,:]
                
                for k_cluster in range(self.K):
                    #print('k:',k_cluster)
                    # get the centroid coordinates
                    mu = self.mu[k_cluster, :]
                    #print('mu:',mu)
                    
                    # now the distance sample-centroid
                    if (self.metric == 'euclidean'):
                        dist = linalg.norm(x-mu)**2
                    elif (self.metric == 'L1'):
                        #print('x:',x)
                        #print('mu:', mu)
                        dist = L1(x,mu)
                        #print('dist:',dist)
                    
                    distances_sample.append(dist)
                    
                index = distances_sample.index(min(distances_sample))
                
                self.C[index].append(i_data)
             
            #Now compute the new centroids
            mu_new=[]
            SE = []
            for k_cluster in range(self.K):
                data_k = [data[i] for i in self.C[k_cluster]]
                #print('data_k:', data_k)
                mu_k = np.average(data_k, axis=0)
                #print('mu_k', mu_k)
                mu_new.append(mu_k)
                SE.append(linalg.norm(self.mu[k_cluster,:] - mu_k)**2)
                 
            error = np.sum(SE)
            print('error:',error)
            self.errors.append(error)
             # breaking condition
                
            if ( error < self.tol):
                self.is_fitted = True
                return self
                break
            else:
                #nope....
                for k_cluster in range(self.K):
                    self.mu[k_cluster,:] = mu_new[k_cluster]
                    
                
    def predict(self,data):
        
        # Check if Kmeans has been trained
        if (not self.is_fitted):
            print('Error:  Kmeans not fitted yet')
            return
        
        # initialize the prediction vector
        y_pred = np.zeros(data.shape[0])
        
        # Loop over data
        for i in range(data.shape[0]):
            distances=[]
            x = data[i,:]
            
            #for each data-sample loop over clusters
            for k_cluster in range(self.K):
                mu = self.mu[k_cluster,:]
                if (self.metric == 'euclidean'):
                    dist = linalg.norm(x-mu)**2
                elif (self.metric == 'L1'):
                    dist = L1(np.array(x),np.array(mu))
                    
                distances.append(dist)
            
            index = distances.index(min(distances))  
            y_pred[i] = index
            
        return y_pred
        
        
        

In [9]:
kmeans = KMeans_custom(metric='euclidean', tol = 0.0001, random_seed=42)

In [10]:
kmeans.fit(data=X_train_std,num_clusters=8)

error: 109.20959859
error: 2.22796683252
error: 0.307872200335
error: 0.238602701491
error: 0.184798397386
error: 0.0283492351708
error: 0.0660470390755
error: 0.0511914704929
error: 0.0215992310348
error: 0.0


In [11]:
#X_test_std = sc.transform(X_test)
y_custom = kmeans.predict(X_train_std)

## 3. Comparing with sklearn KMeans

In [12]:
# Let's try with sklearn
from sklearn.cluster import KMeans
km = KMeans(n_clusters=8, init = 'random', n_init=10, max_iter = 300, tol = 1e-4, random_state = 0)
y_km = km.fit_predict(X_train_std)

In [13]:
from sklearn.metrics import confusion_matrix
confmat = confusion_matrix(y_true=y_km, y_pred=y_custom)
print('confmat = \n', confmat)

confmat = 
 [[11  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 10  0]
 [ 0  0  0  0  0 15  0  0]
 [ 0  0  3 23  0  0  0  0]
 [ 0  8  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0]
 [ 0  0  0  0 24  0  0  0]
 [ 0  0  0  0  0  0  0  1]]


In [14]:
km.cluster_centers_

array([[ 0.9815525 ,  1.90311473, -0.10050378,  0.        , -0.52543521,
        -0.09271456, -0.65757419],
       [-1.41356097, -0.43996198, -0.10050378,  0.        , -0.09451406,
         0.59020115,  0.68458342],
       [ 0.11556212,  0.15791841, -0.10050378,  0.        , -0.34703254,
        -0.35480121, -0.49645118],
       [-0.96565339, -0.80838828, -0.10050378,  0.        , -0.06176991,
        -1.046986  , -0.66261699],
       [ 0.7927338 ,  1.0287442 , -0.10050378,  0.        ,  1.87513011,
        -0.35054793, -0.78049241],
       [ 0.11875444, -0.50494898, -0.10050378,  0.        , -2.3094619 ,
        -0.41666716, -0.73361959],
       [ 0.82465704, -0.17622308, -0.10050378,  0.        ,  0.39909945,
         1.31403274,  1.40690547],
       [-0.01532315,  0.6388222 ,  9.94987437,  0.        ,  0.50431548,
         1.01243624,  1.20859742]])

In [15]:
kmeans.mu

array([[ 0.9815525 ,  1.90311473, -0.10050378,  0.        , -0.52543521,
        -0.09271456, -0.65757419],
       [ 0.7927338 ,  1.0287442 , -0.10050378,  0.        ,  1.87513011,
        -0.35054793, -0.78049241],
       [-0.18890576, -0.57156066, -0.10050378,  0.        , -1.81344345,
        -0.72174362, -0.99081912],
       [-1.00008564, -0.82479804, -0.10050378,  0.        ,  0.05887915,
        -1.02308795, -0.56389508],
       [ 0.82465704, -0.17622308, -0.10050378,  0.        ,  0.39909945,
         1.31403274,  1.40690547],
       [ 0.11556212,  0.15791841, -0.10050378,  0.        , -0.34703254,
        -0.35480121, -0.49645118],
       [-1.41356097, -0.43996198, -0.10050378,  0.        , -0.09451406,
         0.59020115,  0.68458342],
       [-0.01532315,  0.6388222 ,  9.94987437,  0.        ,  0.50431548,
         1.01243624,  1.20859742]])